In [64]:
import sys
import os
import timm
import torch
from timm.models import create_model
import torch.nn as nn

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from R_MeeTo import RMeeTo_Mamba,RMeeTo_Mamba_Video,RMeeToVisionTransformer
#visualization
import random
import torchvision
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from PIL import Image
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from utils.visualization import make_visualization
import matplotlib.pyplot as plt

In [65]:
#choose the model
model_merge = 'RMeeTo_small'
model_pth = './model_best.pth'

In [66]:
#set merge parameters
merge_para = {
        'num_prune': 11,
        'distance': 'cosine',
        'metric': 'X',
        'merge_interval': 2,
        'if_prune': False,
        'if_order': True,
        'class_token': True,
        'if_merge': True,
        'if_merge_odd': False,
        'merge_mode':'sum',
        'choose':'max',
        'compare': 11,
    }

print("Merge para:",merge_para)

In [67]:
model = create_model(
    model_merge,
    pretrained=False,
    model_pth=model_pth,
    merge_para=merge_para,
    if_visualize=True,
)
model.load_state_dict(torch.load(model_pth)['model'])
device = 'cuda'
model.to(device)

In [68]:
input_size = 224
transform_list = [
transforms.Resize(int((256 / 224) * input_size), interpolation=InterpolationMode.BICUBIC),
transforms.CenterCrop(input_size)]   
mean = IMAGENET_DEFAULT_MEAN
std = IMAGENET_DEFAULT_STD
transform_vis  = transforms.Compose(transform_list)
transform_norm = transforms.Compose(transform_list + [
transforms.ToTensor(),
transforms.Normalize(mean, std)])

img = Image.open('./test_figs/ILSVRC2012_val_00005225.JPEG')

img_vis = transform_vis(img)
img_norm = transform_norm(img)
img_norm = img_norm.unsqueeze(0).to(device)
out = model(img_norm)

source = model.source
print(f"{source.shape[1]} tokens at the end")
vision_picture=make_visualization(img_vis, source, patch_size=16, class_token=True)
#show the visualization
plt.imshow(vision_picture)
plt.show()
